In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


# TEST

In [1]:
    from utils.load_Datasets import load_Datasets
    from utils.load_Model import load_Model
    from utils.utils import *


    dataset_name='pile_uncopyrighted_parquet'
    dataset_subset=[0,1]
    dataset = load_Datasets(dataset_config_dict={
                'name': dataset_name,
                'subset': dataset_subset,
                'base_directory': None
            })
    print(f"Original dataset size: {len(dataset)}")
    def filter_item(index, item, tokenizer, tokens_min_length):
        tokens = tokenizer(item['text'], truncation=False, return_tensors='pt')
        if tokens['input_ids'].shape[1] >= tokens_min_length:
            return index
        return None

    def filter_dataset_indices(dataset, tokenizer, tokens_min_length=1024, max_workers=4):
        valid_indices = []
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(filter_item, i, item, tokenizer, tokens_min_length): i for i, item in enumerate(dataset)}
            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                if result is not None:
                    valid_indices.append(result)
        valid_indices.sort()
        return valid_indices

    valid_indices = filter_dataset_indices(dataset, tokenizer, tokens_min_length=tokens_min_length, max_workers=8)
    dataset = dataset.select(valid_indices)
    print(f"Filtered dataset size: {len(dataset)}")

Original dataset size: 182759


In [17]:
dataset[10]

{'text': '{\n  "fpsLimit": 60,\n  "preset": "basic",\n  "background": {\n    "color": "#0d47a1",\n    "image": "",\n    "position": "50% 50%",\n    "repeat": "no-repeat",\n    "size": "cover"\n  }\n}',
 'meta': ['Github'],
 'id': '0-10'}

In [18]:
import subprocess
from datetime import datetime

# Define the hyperparameters in Python
model_name = "olmo-1b"
dataset_name = 'pile_uncopyrighted_parquet' #"openwebtext_parquet"
dataset_subset = [0,1]  # Example list of integers
tokens_min_length = 1024

model_checkpoints = [
    "main", 
    "step100000-tokens419B", 
    "step20000-tokens84B", 
    "step350000-tokens1468B", 
    "step400000-tokens1678B", 
    "step450000-tokens1887B", 
    "step50000-tokens210B", 
    "step500000-tokens2097B", 
    "step550000-tokens2307B", 
    "step600000-tokens2517B", 
    "step650000-tokens2726B", 
    "step700000-tokens2936B"
]
#model_checkpoints =model_checkpoints[:1] 
# Convert dataset_subset list to a string format
dataset_subset_str = ' '.join(map(str, dataset_subset))
dataset_subset_str_bar = '_'.join(map(str, dataset_subset))


# Function to create and submit a job
def submit_job(model_checkpoint):
    # Generate current timestamp
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Create dynamic session_name
    session_name = f"{current_time}_{model_name}_{model_checkpoint}_{dataset_name}_{dataset_subset_str_bar}"
    print(session_name)
    sbatch_script = f"""#!/bin/bash

#SBATCH --job-name={model_checkpoint}
#SBATCH --time=72:10:00
#SBATCH --output=LOG/%x-%j.out
#SBATCH --error=LOG/%x-%j.err

#SBATCH -N 1
#SBATCH -c 20
#SBATCH --gres=gpu:volta:1
#SBATCH --mem-per-cpu=15G

echo "Model checkpoint: {model_checkpoint}"
echo "CPUs per task: $SLURM_CPUS_PER_TASK"
module load anaconda/2023a-pytorch


python Cacheing_OLMo_1B.py --session_name {session_name} --model_name {model_name} --model_checkpoint {model_checkpoint} --dataset_name {dataset_name} --dataset_subset {dataset_subset_str}  --tokens_min_length {tokens_min_length}
"""

    script_filename = f"LLSubScripts/submit_job_{model_checkpoint}.sbatch"
    
    with open(script_filename, "w") as f:
        f.write(sbatch_script)

    # Submit the job
    subprocess.run(["sbatch", script_filename])

# Submit a job for each model checkpoint
for checkpoint in model_checkpoints:
    submit_job(checkpoint)


20240609_111233_olmo-1b_main_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219380
20240609_111233_olmo-1b_step100000-tokens419B_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219381
20240609_111233_olmo-1b_step20000-tokens84B_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219382
20240609_111233_olmo-1b_step350000-tokens1468B_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219383
20240609_111233_olmo-1b_step400000-tokens1678B_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219384
20240609_111233_olmo-1b_step450000-tokens1887B_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219385
20240609_111233_olmo-1b_step50000-tokens210B_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219386
20240609_111233_olmo-1b_step500000-tokens2097B_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219387
20240609_111233_olmo-1b_step550000-tokens2307B_pile_uncopyrighted_parquet_0_1
Submitted batch job 26219388
20240609_111234_olmo-1b_step600000-tokens2517B_pile_uncopyrig

NameError: name 'model' is not defined

tensor([[26170, 14053,   310,   209, 50277, 50279]])

In [ ]:
os.listdir(os.path.expanduser("~")+'/ML/Models/OLMo-1B/'), os.listdir(os.path.expanduser("~")+'/ML/Models/OLMo-7B/')

(['step20000-tokens84B',
  'step600000-tokens2517B',
  'step50000-tokens210B',
  'step350000-tokens1468B',
  'step550000-tokens2307B',
  'step700000-tokens2936B',
  'step450000-tokens1887B',
  '.ipynb_checkpoints',
  'step400000-tokens1678B',
  'step500000-tokens2097B',
  '.git',
  'step100000-tokens419B',
  '.gitattributes',
  'step650000-tokens2726B',
  'main'],
 ['step450000-tokens1991B',
  'step350000-tokens1548B',
  'step250000-tokens1106B',
  'step50000-tokens221B',
  'step20000-tokens88B',
  'step400000-tokens1769B',
  'step150000-tokens664B',
  'step30000-tokens133B',
  'step300000-tokens1327B',
  'step550000-tokens2433B',
  '.ipynb_checkpoints',
  'step10000-tokens44B',
  '.git',
  'step500000-tokens2212B',
  'step200000-tokens885B',
  'step40000-tokens177B',
  '.gitattributes',
  'main',
  'step100000-tokens442B'])

In [ ]:


model_path=os.path.expanduser("~")+'/ML/Models/OLMo-1B/'+'step600000-tokens2517B'
config_path=model_path+'/config.json'
config=AutoConfig.from_pretrained(config_path)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)
for param in model.parameters():
    param.requires_grad = False
model=model.to(torch.device("cuda"))
model.eval()


OLMoForCausalLM(
  (model): OLMo(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 2048)
      (emb_drop): Dropout(p=0.0, inplace=False)
      (ln_f): LayerNorm()
      (blocks): ModuleList(
        (0-15): 16 x OLMoSequentialBlock(
          (dropout): Dropout(p=0.0, inplace=False)
          (act): SwiGLU()
          (attn_out): Linear(in_features=2048, out_features=2048, bias=False)
          (ff_out): Linear(in_features=8192, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
          (attn_norm): LayerNorm()
          (ff_norm): LayerNorm()
          (att_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (ff_proj): Linear(in_features=2048, out_features=16384, bias=False)
        )
      )
    )
  )
)

In [ ]:
hook_outputs = {}

def hook_fn(module, input, output):
    #print(f"Hook on {module.__class__.__name__}")
    hook_outputs[module] = output
    return output

# Register hooks on the specified layers
def register_hooks(model):
    hooks = []
    layers = model.model.transformer.blocks
    
    # Register hooks on the 2nd, 3rd, and last layers
    layers_to_hook = [1, 2, len(layers) - 1]
    
    for layer_idx in layers_to_hook:
        layer = layers[layer_idx]
        
        # Register hooks on the feed-forward network output (ff_out)
        hooks.append(layer.ff_out.register_forward_hook(hook_fn))
    
    return hooks

model=olmo

hooks = register_hooks(model)

message = ["Language modeling is "]
inputs = tokenizer(message, return_tensors='pt', return_token_type_ids=False).to(torch.device("cuda"))
with torch.no_grad():
    response = model.generate(**inputs, max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)
    print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])
for hook in hooks:
    hook.remove()

Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Li

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

OLMoForCausalLM(
  (model): OLMo(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 2048)
      (emb_drop): Dropout(p=0.0, inplace=False)
      (ln_f): LayerNorm()
      (blocks): ModuleList(
        (0-15): 16 x OLMoSequentialBlock(
          (dropout): Dropout(p=0.0, inplace=False)
          (act): SwiGLU()
          (attn_out): Linear(in_features=2048, out_features=2048, bias=False)
          (ff_out): Linear(in_features=8192, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
          (attn_norm): LayerNorm()
          (ff_norm): LayerNorm()
          (att_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (ff_proj): Linear(in_features=2048, out_features=16384, bias=False)
        )
      )
    )
  )
)

In [ ]:
                      
cache = {}

def hook_fn(module, input, output):
    # Retrieve the full name from the context in which the hook was registered
    full_name = hook_fn.full_names.get(module, "Unknown")  
    if full_name not in cache:
        cache[full_name] = output.detach()  # Keep tensor on CUDA
    else:
        # Concatenate new output tensor along axis=1 with previous tensors, keep on CUDA
        cache[full_name] = torch.cat((cache[full_name], output.detach()), dim=1)

def register_hooks(model, module_list):
    hooks = []
    hook_fn.full_names = {}  # Dictionary to store full names accessible by module object
    for name, module in model.named_modules():
        if name in module_list:
            hooks.append(module.register_forward_hook(hook_fn))
            hook_fn.full_names[module] = name
    return hooks

# Setup for model and tokenizer
model = olmo  # Ensure model is correctly assigned
module_list = ["model.transformer.blocks.15.attn_norm", "model.transformer.blocks.15.ff_proj"]
hooks = register_hooks(model, module_list)

# Example text generation
message = ["Language modeling is "]
inputs = tokenizer(message, return_tensors='pt', return_token_type_ids=False).to(device)
with torch.no_grad():
    response = model.generate(**inputs, max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)
    print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

# Remove hooks after use
for hook in hooks:
    hook.remove()

# Move to CPU after all GPU computations if needed
for key in cache:
    cache[key] = cache[key].cpu()  # Now move tensors to CPU


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument tensors in method wrapper_CUDA_cat)

In [ ]:
model()

OLMoForCausalLM(
  (model): OLMo(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 2048)
      (emb_drop): Dropout(p=0.0, inplace=False)
      (ln_f): LayerNorm()
      (blocks): ModuleList(
        (0-15): 16 x OLMoSequentialBlock(
          (dropout): Dropout(p=0.0, inplace=False)
          (act): SwiGLU()
          (attn_out): Linear(in_features=2048, out_features=2048, bias=False)
          (ff_out): Linear(in_features=8192, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
          (attn_norm): LayerNorm()
          (ff_norm): LayerNorm()
          (att_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (ff_proj): Linear(in_features=2048, out_features=16384, bias=False)
        )
      )
    )
  )
)

In [ ]:

tensor = cache['model.transformer.blocks.15.attn_norm']
if tensor.is_cuda:
    print("Tensor is on CUDA device:", tensor.get_device())
else:
    print("Tensor is on CPU")

Tensor is on CPU
